In [560]:
using Pkg
Pkg.activate("CT")
using CT

  Activating project at `~/CT_MPS_mini/CT`


In [561]:
Pkg.status()

Project CT v0.1.0
Status `~/CT_MPS_mini/CT/Project.toml`
  [c7e460c6] ArgParse v1.2.0
⌃ [6e4b80f9] BenchmarkTools v1.5.0
⌅ [9136182c] ITensors v0.6.16
  [682c06a0] JSON v0.21.4
⌃ [33e6dc65] MKL v0.7.0
⌃ [91a5bcdd] Plots v1.40.5
⌃ [9f0aa9f4] TCIITensorConversion v0.1.4
⌃ [b261b2ec] TensorCrossInterpolation v0.9.10
⌃ [a759f4b9] TimerOutputs v0.5.24
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random
Info Packages marked with ⌃ and ⌅ have new versions available. Those with ⌃ may be upgradable, but those with ⌅ are restricted by compatibility constraints from upgrading. To see why use `status --outdated`


In [562]:
# benchmark adder_MPO
using ITensors
include("CT/src/test_adder_MPO.jl")
# initialize random state
using .CT: _initialize_basis, _initialize_vector, P_MPO, XI_MPO, I_MPO, adder_MPO
using Random
L = 4
ancilla = 0
folded = true
seed_vec = 123457
xj = Set([1//3, 2//3])
i1 = 1
p_ctrl = 1.0
p_proj = 0.0
_maxdim = 50
_cutoff = 1e-15
seed = 123457
x0 = nothing
qubit_site, ram_phy, phy_ram, phy_list = _initialize_basis(L, ancilla, folded)
println(phy_list)
println(ram_phy)
println(phy_ram)
rng = MersenneTwister(seed_vec)
rng_vec = seed_vec === nothing ? rng : MersenneTwister(seed_vec)
initial_state = _initialize_vector(L, ancilla, x0, folded, qubit_site, ram_phy, phy_ram, phy_list, rng_vec, _cutoff, _maxdim)
# println(initial_state)

[1, 2, 3, 4]
[1, 4, 2, 3]
[1, 3, 4, 2]


MPS
[1] ((dim=2|id=826|"Qubit,Site,n=1"), (dim=8|id=949|"Link,l=1"))
[2] ((dim=8|id=949|"Link,l=1"), (dim=2|id=702|"Qubit,Site,n=2"), (dim=4|id=157|"Link,l=2"))
[3] ((dim=4|id=157|"Link,l=2"), (dim=2|id=379|"Qubit,Site,n=3"), (dim=2|id=332|"Link,l=3"))
[4] ((dim=2|id=332|"Link,l=3"), (dim=2|id=644|"Qubit,Site,n=4"))


In [563]:
# # debug 
# # haining
# using .CT: add1, power_mpo
# add1_mpo=MPO(add1(i1,L,phy_ram,phy_list),qubit_site)
# add1_6,add1_3=power_mpo(add1_mpo,[div(2^L,6)+1,div(2^L,3)])

# # tao
# # Convert fractions to binary shifts
# shift_1_6_bits, shift_1_6_amount = fraction_to_binary_shift(1, 6, L)
# shift_1_3_bits, shift_1_3_amount = fraction_to_binary_shift(1, 3, L)
# println("shift_1_6_bits: ", shift_1_6_bits)
# println("shift_1_3_bits: ", shift_1_3_bits)
# # println("shift_1_6_amount: ", shift_1_6_amount)
# # println("shift_1_3_amount: ", shift_1_3_amount)
# # Create binary addition MPOs
# # add_mpo_1_6 = create_binary_addition_mpo(shift_1_6_bits, qubit_site, phy_ram, phy_list)
# add_mpo_1_3 = create_binary_addition_mpo([0,0,0,1], qubit_site, phy_ram, phy_list, ram_phy)

# basis_state = productMPS(qubit_site, [1,2,2,1])
# # println(basis_state)
# # println(add_mpo_1_3)
# result = basis_state * add_mpo_1_3
# print_mps_as_binary(result, L)
# # println("result: ", vec(array(contract(result))))

# # result_vec = vec(array(contract(result)))
# # print(vector(result_vec))
# # println("~"^100)
# # n = mps_to_computational_basis(result)
# # binary_str = string(n, base=2, pad=L)  # L is the number of qubits
# # println("State: |$binary_str⟩")
# # For your specific MPS structure
# # Contract carry_phy_2 (tensors 1 and 3)


In [ ]:
using ITensors
include("CT/src/test_adder_MPO.jl")
using .CT: _initialize_basis, _initialize_vector, P_MPO, XI_MPO, I_MPO, adder_MPO
using Random

using SparseArrays

qubit_site, ram_phy, phy_ram, phy_list = _initialize_basis(4, 0, folded)


function create_msb_ordered_tensor(dense_mat, L)
    """Create tensor with MSB-first bit ordering instead of default reshape"""
    tensor_shape = ntuple(i -> 2, 2*L)
    A_tensor = zeros(Int, tensor_shape)
    
    # Fill tensor manually with MSB-first ordering
    for i in 0:2^L-1
        for j in 0:2^L-1
            if dense_mat[i+1, j+1] != 0
                # Convert i,j to MSB-first bit indices
                i_str = string(i, base=2, pad=L)
                j_str = string(j, base=2, pad=L)
                i_bits = [parse(Int, c) + 1 for c in i_str]  # MSB first, 1-indexed
                j_bits = [parse(Int, c) + 1 for c in j_str]  # MSB first, 1-indexed
                
                A_tensor[i_bits..., j_bits...] = dense_mat[i+1, j+1]
            end
        end
    end
    
    return A_tensor
end

function mpo_adder_1_3(L::Int, qubit_site::Vector{Index{Int64}}, ram_phy::Vector{Int}, phy_ram::Vector{Int}, phy_list::Vector{Int}, folded::Bool)
    coordinates = []
    shift_str = "01"^L
    shift_num = parse(Int, shift_str, base=2)
    for i in 0:2^L-1
        # println("phys order ", i, "->", (i+5) % (2^L))
        # println("phys order binary ", string(i, base=2, pad=L), "->", string((i+5) % (2^L), base=2, pad=L))
        # println("ram order binary ", string(string(i, base=2, pad=L)[ram_phy]), "->", string(string((i+5) % (2^L), base=2, pad=L)[ram_phy]))
        # println("ram order ", parse(Int, string(string(i, base=2, pad=L)[ram_phy]), base=2), "->", parse(Int, string(string((i+5) % (2^L), base=2, pad=L)[ram_phy]), base=2))
        coordinate = (parse(Int, string(string((i+shift_num) % (2^L), base=2, pad=L)[ram_phy]), base=2)+1, parse(Int, string(string(i, base=2, pad=L)[ram_phy]), base=2)+1)
        push!(coordinates, coordinate)
    end
    # println(coordinates)
    matrix_size = (2^L, 2^L)  # or whatever size you need

    # Method 2: Direct dense matrix construction
    dense_mat = zeros(Int, matrix_size)
    for (i, j) in coordinates
        dense_mat[i, j] = 1
    end
    println(dense_mat)

    # Reshape 16x16 matrix into 4-qubit tensor with 8 indices
    # Each qubit has dimension 2, so we have (2,2,2,2) ⊗ (2,2,2,2)
    A_tensor = create_msb_ordered_tensor(dense_mat, L)
    # println(A_tensor[1,1,2,1,2,2,2,2])
    # Set small values to zero
    A_tensor[abs.(A_tensor) .< 1e-10] .= 0

    # Create ITensor with both input and output indices
    # Input indices (ket): sites
    # Output indices (bra): sites'
    A_itensor = ITensor(A_tensor, qubit_site'..., qubit_site...)

    # Convert to MPO (Matrix Product Operator)
    cutoff = 1e-12
    maxdim = 100

    # Create MPO from the operator ITensor
    mpo = MPO(A_itensor, qubit_site; cutoff=cutoff, maxdim=maxdim)
    orthogonalize!(mpo, length(mpo))

    # get bond dimensions
        # Assuming you have an MPO called `mpo`
    bond_dims = []
    for i in 1:length(mpo)-1
        # Get the common index between sites i and i+1
        common_idx = commonind(mpo[i], mpo[i+1])
        push!(bond_dims, dim(common_idx))
    end
    return mpo, bond_dims, dense_mat, A_tensor
end

mpo, bond_dims, dense_mat, A_tensor = mpo_adder_1_3(4, qubit_site, ram_phy, phy_ram, phy_list, folded);

[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0; 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0; 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1; 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0; 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0; 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]


In [ ]:
mpo, bond_dims, dense_mat, A_tensor = mpo_adder_1_3(4, qubit_site, ram_phy, phy_ram, phy_list, folded);
for i in 1:2, j in 1:2, k in 1:2, l in 1:2
    mps = productMPS([site[2] for site in siteinds(mpo)], [i,j,k,l])
    result = mpo * mps
    # println(fieldnames(typeof(result)))
    println("~"^60)
    println(i-1,j-1,k-1,l-1)
    println(result.data)
    println("~"^60)
end







 

[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0; 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0; 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1; 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0; 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0; 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
0000
ITensor[ITensor ord=2
Dim 1: (dim=1|id=581|"Link,l=1")'
Dim 2: (dim=2|id=101|"Qubit,Site,n=1")'
NDTensors.Dense{Float64, Vector{Float64}}
 1×2
 -1.0000000000000002  0.0, ITensor ord=3
Dim 1: (dim=2|id=770|"Qubit,Site,n=2")'
Dim 2: (dim=1|id=302|"Link,l=2")'
Dim 3: (dim=1|id=581|"Link,l=1")'
NDTensors.Dense{Float64, Vector{Float64}}
 2×1×1
[:, :, 1] =
 0.0
 1.0, ITensor ord=3
Dim 1: (dim=2|id=410|"Qubit

In [565]:
folded = true
tensor_list = Array{Any,1}[]
n = 2
mpo_list = Vector{Any}(undef, 0)
for L in [4, 8]
    qubit_site, ram_phy, phy_ram, phy_list = _initialize_basis(L, 0, folded)
    mpo, bond_dims = mpo_adder_1_3(L, folded);
    push!(mpo_list, mpo)
    println(mpo[n])

    # # println("L: ", L, " bond_dims: ", bond_dims)
    # tensor = Array(mpo[n], inds(mpo[n])...)
    # tensor[abs.(tensor) .< 1e-10] .= 0
    # vec = reshape(tensor, :)
    # println(length(vec))
    # push!(tensor_list, vec)
    # println(mpo[n])
end

# After collecting tensor_list...
for entries in eachrow(hcat(tensor_list...))
    println(entries)
end

# for i in 1:length(tensor_list)
#     for j in i+1:length(tensor_list)
#         diff = tensor_list[i] - tensor_list[j]
#         println(findall(x -> abs(x) >= 1e-10, diff))
#     end
# end

ITensor ord=4
Dim 1: (dim=3|id=755|"Link,n=1")
Dim 2: (dim=2|id=390|"Qubit,Site,n=2")'
Dim 3: (dim=2|id=390|"Qubit,Site,n=2")
Dim 4: (dim=4|id=123|"Link,n=2")
NDTensors.Dense{Float64, Vector{Float64}}
 3×2×2×4
[:, :, 1, 1] =
 0.0                 0.0
 0.0                 0.0
 0.5773502691896258  0.0

[:, :, 2, 1] =
 0.0                 0.0
 0.5773502691896258  0.0
 0.0                 0.5773502691896258

[:, :, 1, 2] =
 0.0  -0.9999999999999999
 0.0   0.0
 0.0   0.0

[:, :, 2, 2] =
 0.0  0.0
 0.0  0.0
 0.0  0.0

[:, :, 1, 3] =
  0.0                    0.0
  0.0                    7.850462293418877e-17
 -7.401486830834377e-17  0.0

[:, :, 2, 3] =
 1.0                    0.0
 3.700743415417188e-17  0.0
 0.0                    3.700743415417188e-17

[:, :, 1, 4] =
 0.0                    0.0
 0.0                    1.0000000000000002
 1.162101865614033e-32  0.0

[:, :, 2, 4] =
 -7.850462293418876e-17    0.0
 -1.1102230246251565e-16   0.0
  0.0                     -1.1102230246251565e-16
IT

In [566]:
mpo_list[1]

MPO
[1] ((dim=2|id=596|"Qubit,Site,n=1")', (dim=2|id=596|"Qubit,Site,n=1"), (dim=3|id=755|"Link,n=1"))
[2] ((dim=3|id=755|"Link,n=1"), (dim=2|id=390|"Qubit,Site,n=2")', (dim=2|id=390|"Qubit,Site,n=2"), (dim=4|id=123|"Link,n=2"))
[3] ((dim=4|id=123|"Link,n=2"), (dim=2|id=940|"Qubit,Site,n=3")', (dim=2|id=940|"Qubit,Site,n=3"), (dim=2|id=599|"Link,n=3"))
[4] ((dim=2|id=599|"Link,n=3"), (dim=2|id=281|"Qubit,Site,n=4")', (dim=2|id=281|"Qubit,Site,n=4"))


In [567]:
ram_phy

8-element Vector{Int64}:
 1
 8
 2
 7
 3
 6
 4
 5

In [568]:
mpo_list[2][1]

ITensor ord=3 (dim=2|id=992|"Qubit,Site,n=1")' (dim=2|id=992|"Qubit,Site,n=1") (dim=3|id=241|"Link,n=1")
NDTensors.Dense{Float64, Vector{Float64}}

In [569]:
array(mpo_list[2][1])

2×2×3 Array{Float64, 3}:
[:, :, 1] =
 -0.707107   0.0
  0.0       -0.707107

[:, :, 2] =
  0.0  0.0
 -1.0  0.0

[:, :, 3] =
 0.0  1.0
 0.0  0.0

In [570]:
mpo_list[2][2]

ITensor ord=4 (dim=3|id=241|"Link,n=1") (dim=2|id=812|"Qubit,Site,n=2")' (dim=2|id=812|"Qubit,Site,n=2") (dim=4|id=542|"Link,n=2")
NDTensors.Dense{Float64, Vector{Float64}}

In [571]:
array(mpo_list[2][2])[1,:,:,4]

2×2 Matrix{Float64}:
 0.0  0.0
 0.0  0.0

In [572]:
array(mpo_list[2][2])[2,:,:,1]

2×2 Matrix{Float64}:
 0.0  0.57735
 0.0  0.0

In [573]:
mpo_list[2][2]

ITensor ord=4 (dim=3|id=241|"Link,n=1") (dim=2|id=812|"Qubit,Site,n=2")' (dim=2|id=812|"Qubit,Site,n=2") (dim=4|id=542|"Link,n=2")
NDTensors.Dense{Float64, Vector{Float64}}

In [574]:
mpo_list[2][3]

ITensor ord=4 (dim=4|id=542|"Link,n=2") (dim=2|id=411|"Qubit,Site,n=3")' (dim=2|id=411|"Qubit,Site,n=3") (dim=4|id=836|"Link,n=3")
NDTensors.Dense{Float64, Vector{Float64}}

In [575]:
array(mpo_list[2][3]) 

4×2×2×4 Array{Float64, 4}:
[:, :, 1, 1] =
 0.0   0.0
 0.0  -0.632456
 0.0   0.0
 0.0   0.0

[:, :, 2, 1] =
 -0.774597  0.0
  0.0       0.0
  0.0       0.0
  0.0       0.0

[:, :, 1, 2] =
 0.0   0.0
 0.0   0.0
 0.0  -0.816497
 0.0   0.0

[:, :, 2, 2] =
  0.0      0.0
  0.0      0.0
  0.0      0.0
 -0.57735  0.0

[:, :, 1, 3] =
 -0.707107  0.0
  0.0       0.0
  0.0       0.0
  0.0       0.0

[:, :, 2, 3] =
 0.0  -0.707107
 0.0   0.0
 0.0   0.0
 0.0   0.0

[:, :, 1, 4] =
  0.0       0.0
  0.0       0.0
  0.0       0.0
 -0.707107  0.0

[:, :, 2, 4] =
 0.0   0.0
 0.0   0.0
 0.0   0.0
 0.0  -0.707107

In [576]:

array(mpo_list[2][2]) - array(mpo_list[2][3])

LoadError: DimensionMismatch: dimensions must match: a has dims (Base.OneTo(3), Base.OneTo(2), Base.OneTo(2), Base.OneTo(4)), b has dims (Base.OneTo(4), Base.OneTo(2), Base.OneTo(2), Base.OneTo(4)), mismatch at 1

In [ ]:
sqrt(5)

2.23606797749979

In [ ]:
size(mpo_,1)

3

In [ ]:
mpo_ = array(mpo_list[1][3])
for left in 1:size(mpo_,1)
    for right in 1:size(mpo_,4)
        t_ = mpo_[left,:,:,right]
        sum_ = sum(abs.(t_))
        if sum(abs.(t_)) > 1e-10
            println(left, " ", right)
        end
    end
end

1 1
2 2
3 1
4 2


In [ ]:
prod(mpo_list[1])

ITensor ord=8 (dim=2|id=285|"Qubit,Site,n=1")' (dim=2|id=285|"Qubit,Site,n=1") (dim=2|id=872|"Qubit,Site,n=2")' (dim=2|id=872|"Qubit,Site,n=2") (dim=2|id=655|"Qubit,Site,n=3")' (dim=2|id=655|"Qubit,Site,n=3") (dim=2|id=438|"Qubit,Site,n=4")' (dim=2|id=438|"Qubit,Site,n=4")
NDTensors.Dense{Float64, Vector{Float64}}

In [ ]:
mpo_ = array(mpo_list[1][4])
for left in 1:size(mpo_,1)
    t_ = mpo_[left,:,:]
    sum_ = sum(abs.(t_))
    if sum(abs.(t_)) > 1e-10
        println(left, " ")
    end
end

1 
2 


In [ ]:
.632456^2

0.400000591936

In [ ]:
sqrt(3/5)

0.7745966692414834

In [ ]:
array(mpo_list[1][3])[4,:,:,2]

2×2 Matrix{Float64}:
 0.0          -0.57735
 1.52174e-33  -1.50961e-32

In [ ]:
mpo_list[1][1].tensor.storage

12-element NDTensors.Dense{Float64, Vector{Float64}}:
 -0.7071067811865472
  0.0
  0.0
 -0.7071067811865475
  0.0
  1.0
  0.0
  0.0
  0.0
  0.0
  1.0
  0.0

In [ ]:
NDTensors.Dense(mpo_list[1][1].tensor.storage)

12-element NDTensors.Dense{Float64, NDTensors.Dense{Float64, Vector{Float64}}}:
 -0.7071067811865472
  0.0
  0.0
 -0.7071067811865475
  0.0
  1.0
  0.0
  0.0
  0.0
  0.0
  1.0
  0.0

In [ ]:
(1. .- (mpo_list[1][1].tensor.storage .==0))

12-element Vector{Float64}:
 1.0
 0.0
 0.0
 1.0
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0
 1.0
 0.0

In [ ]:
mpo_list[1][1].tensor.storage = NDTensors.Dense(1. .- (mpo_list[1][1].tensor.storage .==0))

LoadError: setfield!: immutable struct of type Tensor cannot be changed

In [ ]:
mpo_list[1][1]

ITensor ord=3 (dim=2|id=285|"Qubit,Site,n=1")' (dim=2|id=285|"Qubit,Site,n=1") (dim=3|id=685|"Link,n=1")
NDTensors.Dense{Float64, Vector{Float64}}

In [ ]:
mm = MPO(4)

MPO
#undef
#undef
#undef
#undef


In [ ]:
for idx in 1:length(mm)
    mm[idx] = (sign.(round.(abs.((mpo_list[1][idx])),digits=5)))
    
end

In [ ]:
mm

MPO
[1] ((dim=2|id=285|"Qubit,Site,n=1")', (dim=2|id=285|"Qubit,Site,n=1"), (dim=3|id=685|"Link,n=1"))
[2] ((dim=3|id=685|"Link,n=1"), (dim=2|id=872|"Qubit,Site,n=2")', (dim=2|id=872|"Qubit,Site,n=2"), (dim=4|id=464|"Link,n=2"))
[3] ((dim=4|id=464|"Link,n=2"), (dim=2|id=655|"Qubit,Site,n=3")', (dim=2|id=655|"Qubit,Site,n=3"), (dim=2|id=994|"Link,n=3"))
[4] ((dim=2|id=994|"Link,n=3"), (dim=2|id=438|"Qubit,Site,n=4")', (dim=2|id=438|"Qubit,Site,n=4"))


In [ ]:
sum(abs.(prod(mm)-prod(mpo_list[1])))

1.3357409176571358e-14

In [ ]:
mm

MPO
[1] ((dim=2|id=285|"Qubit,Site,n=1")', (dim=2|id=285|"Qubit,Site,n=1"), (dim=3|id=685|"Link,n=1"))
[2] ((dim=3|id=685|"Link,n=1"), (dim=2|id=872|"Qubit,Site,n=2")', (dim=2|id=872|"Qubit,Site,n=2"), (dim=4|id=464|"Link,n=2"))
[3] ((dim=4|id=464|"Link,n=2"), (dim=2|id=655|"Qubit,Site,n=3")', (dim=2|id=655|"Qubit,Site,n=3"), (dim=2|id=994|"Link,n=3"))
[4] ((dim=2|id=994|"Link,n=3"), (dim=2|id=438|"Qubit,Site,n=4")', (dim=2|id=438|"Qubit,Site,n=4"))


In [604]:
qubit_site 

4-element Vector{Index{Int64}}:
 (dim=2|id=101|"Qubit,Site,n=1")
 (dim=2|id=770|"Qubit,Site,n=2")
 (dim=2|id=410|"Qubit,Site,n=3")
 (dim=2|id=853|"Qubit,Site,n=4")

In [ ]:
[site[2] for site in siteinds(mm)]

4-element Vector{Index{Int64}}:
 (dim=2|id=285|"Qubit,Site,n=1")
 (dim=2|id=872|"Qubit,Site,n=2")
 (dim=2|id=655|"Qubit,Site,n=3")
 (dim=2|id=438|"Qubit,Site,n=4")

In [603]:
mps = productMPS([site[2] for site in siteinds(mm)], [1,2,1,2])
# qubit_site

MPS
[1] ((dim=2|id=285|"Qubit,Site,n=1"), (dim=1|id=976|"Link,l=1"))
[2] ((dim=1|id=976|"Link,l=1"), (dim=2|id=872|"Qubit,Site,n=2"), (dim=1|id=291|"Link,l=2"))
[3] ((dim=1|id=291|"Link,l=2"), (dim=2|id=655|"Qubit,Site,n=3"), (dim=1|id=33|"Link,l=3"))
[4] ((dim=1|id=33|"Link,l=3"), (dim=2|id=438|"Qubit,Site,n=4"))


In [ ]:
array(prod(mps))

2×2×2×2 Array{Float64, 4}:
[:, :, 1, 1] =
 0.0  0.0
 0.0  0.0

[:, :, 2, 1] =
 0.0  0.0
 0.0  0.0

[:, :, 1, 2] =
 0.0  1.0
 0.0  0.0

[:, :, 2, 2] =
 0.0  0.0
 0.0  0.0

In [ ]:
array(prod(apply(mm,mps)))

2×2×2×2 Array{Float64, 4}:
[:, :, 1, 1] =
 0.0  0.0
 0.0  0.0

[:, :, 2, 1] =
 0.0  1.0
 0.0  0.0

[:, :, 1, 2] =
 0.0  0.0
 0.0  0.0

[:, :, 2, 2] =
 0.0  0.0
 0.0  0.0

In [ ]:
 prod(apply(mm,mps))[1,2,2,1]

1.0

In [580]:
function compare_tensors_side_by_side()
    folded = true
    tensor_list = []
    L_values = [4, 8, 12]
    n = 2  # Which tensor position to compare
    
    # Collect all tensors
    for L in L_values
        qubit_site, ram_phy, phy_ram, phy_list = _initialize_basis(L, 0, folded)
        mpo, bond_dims = mpo_adder_1_3(L, folded)
        
        tensor = Array(mpo[n], inds(mpo[n])...)
        tensor[abs.(tensor) .< 1e-10] .= 0
        push!(tensor_list, tensor)
    end
    
    # Find maximum tensor size for padding
    max_length = maximum(length.(tensor_list))
    
    # Reshape all tensors to vectors and pad if necessary
    flattened_tensors = []
    for tensor in tensor_list
        flat = reshape(tensor, :)
        # Pad with zeros if needed
        if length(flat) < max_length
            padded = vcat(flat, zeros(max_length - length(flat)))
            push!(flattened_tensors, padded)
        else
            push!(flattened_tensors, flat)
        end
    end
    
    # Print header
    println("Position  ", join([rpad("L=$L", 15) for L in L_values], ""))
    println(repeat("-", 10 + 15 * length(L_values)))
    
    # Print each entry side by side
    for i in 1:max_length
        entries = []
        for j in 1:length(L_values)
            val = flattened_tensors[j][i]
            if abs(val) < 1e-12
                push!(entries, rpad("0", 15))
            else
                push!(entries, rpad(string(round(val, digits=8)), 15))
            end
        end
        println(rpad("[$i]", 10), join(entries, ""))
    end
    
    # Print comparison summary
    println("\n" * repeat("=", 10 + 15 * length(L_values)))
    println("Comparison Summary:")
    
    # Compare pairs
    for i in 1:length(L_values)
        for j in i+1:length(L_values)
            diff = maximum(abs.(flattened_tensors[i] - flattened_tensors[j]))
            if diff < 1e-10
                println("✓ L=$(L_values[i]) vs L=$(L_values[j]): IDENTICAL (max_diff = $diff)")
            else
                println("✗ L=$(L_values[i]) vs L=$(L_values[j]): DIFFERENT (max_diff = $diff)")
            end
        end
    end
    
    return tensor_list, flattened_tensors
end

compare_tensors_side_by_side()

LoadError: BoundsError: attempt to access 4-codeunit String at index [8]